In [2]:
import matplotlib.pyplot as plt
from pennylane import numpy as np
import pennylane as qml
import qiskit
from qiskit.algorithms import QAOA
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from qiskit.algorithms.optimizers import COBYLA, ADAM, SLSQP
from qiskit.utils import QuantumInstance
from qiskit import Aer
from qiskit.opflow import I, X, Y, Z, Zero, One, PauliExpectation, CircuitSampler, StateFn, DictStateFn, CircuitStateFn

In [ ]:
class RandomIsing:
    """
    Stores the information of a model, which is defined by Ising bonds (J) on a 
    graph (G) with a parallel field (h).
    """
    def __init__(self,d,nqubits,localTerm=True):
        """
        d < n 
        d*n even
        """
        self.d = d
        self.nqubits = nqubits
        self.localTerm = localTerm

        self.G = nx.random_regular_graph(d,nqubits)
        self.J = {pair: 2*np.random.randint(2)-1 for pair in list(self.G.edges())}
        
        if localTerm:
            self.h = 2*np.random.randint(2,size=self.nqubits)-1
        else:
            self.h = np.zeros(self.nqubits)

    def drawGraph(self):
        return nx.draw(self.G, with_labels=True, alpha=0.8, node_size=500)

    def setCoupling(self,J):
        for pair in list(self.G.edges()):
            self.J.update({pair:J})

    
    def Hamiltonian(self):
        H = I^self.nqubits
        for pair in list(self.G.edges()):
            H += self.J.get(pair)*((I^pair[0])^Z^(I^(self.nqubits-pair[0]-1)))@((I^pair[1])^Z^(I^(self.nqubits-pair[1]-1)))
        for i in range(self.nqubits):
            H += self.h[i]*((I^i)^Z^(I^(self.nqubits-i-1)))
        H -= I^self.nqubits
        return H.reduce()
    def QAOA(self, p, record=False):
        """returns optimization result of QAOA and the QAOA instance"""
        callback = None
        if record:
            callback = self.callback
        solver = QAOA(COBYLA(maxiter=50000),reps=p,quantum_instance=QuantumInstance(Aer.get_backend('statevector_simulator')),initial_point=np.append(np.linspace(2,0.1,p),np.linspace(0.5,2.5,p)),callback=callback)
        res = solver.compute_minimum_eigenvalue(self.Hamiltonian())
        if res.cost_function_evals >= 50000:
            print('not converged',res)
        return res, solver

In [ ]:
mod=RandomIsing(2,3)
res, solver=mod.QAOA(1)